## Run a job from inside a Domino Workspace

You need a token to invoke a job in Domino. 
- You can use a DOMINO_USER_API_KEY from the workspace environment variable
- You can get a new access token each time you need to make a domino API call. The token is valid for 5 mins

Find the docs for python based client [here](https://github.com/dominodatalab/python-domino) or endpoint based client [here](https://github.com/dominodatalab/python-domino)

In [1]:
import os

In [2]:
import requests
def get_access_token():
    api_proxy=os.environ["DOMINO_API_PROXY"]
    x = requests.get(f"{api_proxy}/access-token")
    if x.status_code==200:
        return x.text
    return ''

In [3]:
##Using API Key
from domino import Domino
# By and large your commands will run against a single project,
# so you must specify the full project name
project_owner=os.environ['DOMINO_PROJECT_OWNER']
project_name=os.environ['DOMINO_PROJECT_NAME']
domino_1 = Domino(f"{project_owner}/{project_name}",api_key=os.environ['DOMINO_USER_API_KEY'])
domino_2 = Domino(f"{project_owner}/{project_name}",auth_token=get_access_token())


In [4]:
##Using API KEY
print("User Id " +  domino_1.get_user_id(project_owner))
##Using Access Token
print("User Id " +  domino_2.get_user_id(project_owner))

ERROR:domino.http_request_manager:{"success":false,"message":"Your role does not authorize you to perform this action","debugMessage":"AnonymousPrincipal is not authorized to perform this action. Anonymous user does not have permission to view user list"}


HTTPError: 403 Client Error: Forbidden for url: http://nucleus-frontend.domino-platform:80/v4/users

### A General Rule of Thumb
1. Use the API Key when invoking the Domino API from outside Domino
2. Use the Access Token when invoking the Domino API key from inside a Domino Workload

## Running Jobs using Domino API from inside a workspace

1. You have to commit your code first. A job is a separate workload
2. Do not pass sensitive information to Jobs. Instead update a user envirionment variable before starting a job and read it from inside a job

In [5]:
import os
import requests
import random

## Passing Env variables
api_host = os.environ['DOMINO_API_HOST']
api_key=os.environ['DOMINO_USER_API_KEY']
url = f"{api_host}/account/setUserEnvironmentVariable"
api_key_headers = {'X-Domino-Api-Key':api_key}

## This endpoint only works with an API Key
request_headers=api_key_headers
request_headers['Content-Type']='application/x-www-form-urlencoded'
a_val = random.randint(0, 100)
b_val = random.randint(0, 100)
print(f"a={a_val} and b={b_val}")
data={'a':str(a_val),'b':str(b_val)}
data={'userEnvironmentVariableName':'a','userEnvironmentVariableValue':str(a_val)}
r = requests.post(url,headers=request_headers,data=data)
print(r.status_code)
data={'userEnvironmentVariableName':'b','userEnvironmentVariableValue':str(b_val)}
r = requests.post(url,headers=request_headers,data=data)
print(r.status_code)

# Go to your user environment variable section in the user settings and confirm that a and b have the same values as shown below

a=28 and b=61
500
500


In [ ]:
## Now we start a job using API Key and go to the jobs manager

In [ ]:
print(domino_1.hardware_tiers_list())

In [ ]:
result = domino_1.runs_start_blocking( ["test.py"], title="Blocking Api key based execution")

In [ ]:
## Now we start a job using access token and go to the jobs manager. In this case we will demonstrate starting a job in non-blocking mode

In [ ]:
domino_2 = Domino(f"{project_owner}/{project_name}",auth_token=get_access_token())
result = domino_1.runs_start( ["test.py"], title="Api key based execution")
print(result)

In [ ]:
run_id=result['runId']
#Remember to create a new instance of Domino everytime you make and api call when using access_token. The access_token is only valid for 5 mins
#You do not need to do this if using an API KEY

#You can manually check the status of the job
domino_2 = Domino(f"{project_owner}/{project_name}",auth_token=get_access_token())
domino_2.runs_status(run_id)